In [1]:
import cv2

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

#cv2.namedWindow("live", cv2.WINDOW_AUTOSIZE); # 命名一個視窗，可不寫
while(True):
    # 擷取影像
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # 彩色轉灰階
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 顯示圖片
    cv2.imshow('live', frame)
    #cv2.imshow('live', gray)

    # 按下 q 鍵離開迴圈
    if cv2.waitKey(1) == ord('q'):
        break

# 釋放該攝影機裝置
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [4]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image

cap = cv2.VideoCapture(0)

def boxSize(arr):
    global data
    box_roll = np.rollaxis(arr,1,0)
    xmax = int(np.amax(box_roll[0]))
    xmin = int(np.amin(box_roll[0]))
    ymax = int(np.amax(box_roll[1]))
    ymin = int(np.amin(box_roll[1]))
    return (xmin,ymin,xmax,ymax)

qrcode = cv2.QRCodeDetector()           

while True:
    ret, frame = cap.read()
    if not ret:
        print("Cannot receive frame")
        break
    img = cv2.resize(frame,(720,420))    
    ok, data, bbox, rectified = qrcode.detectAndDecodeMulti(img) 
    if ok:
        for i in range(len(data)):
            text = data[i]        
            box = boxSize(bbox[i])    
            cv2.rectangle(img,(box[0],box[1]),(box[2],box[3]),(0,0,255),5) 
            print(box[2]-box[0],box[3]-box[1])                  
    cv2.imshow('oxxostudio', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


105 74
110 74
110 76
115 75
112 74
113 75
109 78
113 83
112 85
112 85
113 83
113 86
112 85
114 86
115 84
114 86
114 85
115 85
117 88
117 86
118 86
118 89
118 91
118 89
120 88
119 90
121 90
121 92
120 92
121 92
122 92
122 94
121 92
122 92
122 92
122 92
122 93
123 93
124 95
124 93
124 95
126 96
125 96
126 96
127 98
128 96
128 97
128 98
129 99
128 99
128 97
127 99
126 100
127 100
128 100
128 99
130 98
129 100
130 100
131 100
130 100
131 100
131 100
129 100
130 99
129 98
129 99
129 99
129 99
129 99
129 99
127 99
127 98
127 97
127 97
127 98
127 97
127 97
127 98
127 97
128 96
126 97
126 97
128 99
128 97
129 98
129 97
130 96
130 100
129 99
130 99
129 99
131 98
131 97
131 100
130 99
129 99
130 99
128 98
131 99
130 99
130 99
129 99
129 99
129 99
126 99
132 99
126 99
129 99
130 99
130 99
127 98
128 97
129 99
129 99
131 100
129 99
129 99
128 100
129 99
129 99
129 98
130 100
130 100
130 98
128 97
130 100
130 100
128 96
128 100
130 99
130 99
130 99
130 99
130 100
129 97
127 98
130 100
131 100
131 1

In [ ]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_BRIGHTNESS, 0)
cap.set(cv2.CAP_PROP_CONTRAST, 15)
cap.set(cv2.CAP_PROP_SATURATION, 36)
cap.set(cv2.CAP_PROP_HUE, 0)
cap.set(cv2.CAP_PROP_EXPOSURE, -8)


def find_rect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    # cv2.imshow('Blur', blur)
    a, thresh = cv2.threshold(blur, 150, 255, cv2.THRESH_BINARY_INV)
    # cv2.imshow('Thresh', thresh)
    erode = cv2.erode(thresh, (7, 7), iterations=5)
    # cv2.imshow('Erode', erode)
    dilate = cv2.dilate(erode, (7, 7), iterations=5)
    # cv2.imshow('Dilate', dilate)
    contours, _ = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    cnt_max = max(contours, key=cv2.contourArea)
    return cnt_max

qrcode = cv2.QRCodeDetector()           

while True:
    ret, frame = cap.read()
    if not ret:
        print("Cannot receive frame")
        break
    img = cv2.resize(frame,(720,420))
    ok, data, bbox, rectified = qrcode.detectAndDecodeMulti(img) 
    if ok:
        cnt_max = find_rect(img)
        if cnt_max is not None:
            rect = cv2.minAreaRect(cnt_max)
            
            angle = rect[2]
            # print('Angle:', angle)
            if angle < -45:
                angle = 90 + angle

            (h, w) = img.shape[:2]
            center = (w // 2, h // 2)

            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            
            box = np.int32(cv2.boxPoints(rect))
            
            pts = np.int32(cv2.transform(np.array([box]), M))[0]

            y_min = min(pts[0][0], pts[1][0], pts[2][0], pts[3][0])
            y_max = max(pts[0][0], pts[1][0], pts[2][0], pts[3][0])
            x_min = min(pts[0][1], pts[1][1], pts[2][1], pts[3][1])
            x_max = max(pts[0][1], pts[1][1], pts[2][1], pts[3][1])
            
            # print('x length:', x_max-x_min, ', y length:', y_max-y_min)
            print('data:', data, 'x length:', x_max-x_min, ', y length:', y_max-y_min)
            
            cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
    
    
    cv2.imshow('Final', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


data: ('',) x length: 230 , y length: 170
data: ('',) x length: 231 , y length: 170
data: ('',) x length: 234 , y length: 170
data: ('',) x length: 229 , y length: 170
data: ('',) x length: 228 , y length: 169
data: ('',) x length: 227 , y length: 171
data: ('',) x length: 225 , y length: 169
data: ('',) x length: 223 , y length: 170
data: ('',) x length: 220 , y length: 169
data: ('',) x length: 221 , y length: 169
data: ('',) x length: 218 , y length: 167
data: ('',) x length: 218 , y length: 167
data: ('',) x length: 217 , y length: 166
data: ('',) x length: 216 , y length: 166
data: ('',) x length: 214 , y length: 166
data: ('',) x length: 211 , y length: 167
data: ('',) x length: 205 , y length: 167
data: ('',) x length: 203 , y length: 167
data: ('',) x length: 205 , y length: 166
data: ('',) x length: 162 , y length: 211
data: ('',) x length: 215 , y length: 162
data: ('',) x length: 205 , y length: 163
data: ('',) x length: 199 , y length: 163
data: ('',) x length: 197 , y leng

In [18]:
Video = cv2.VideoCapture(0)  # 读入视频

print('宽:', Video.get(cv2.CAP_PROP_FRAME_WIDTH) )
print('高:', Video.get(cv2.CAP_PROP_FRAME_HEIGHT) )
print('帧率:', Video.get(cv2.CAP_PROP_FPS) )
print('亮度:', Video.get(cv2.CAP_PROP_BRIGHTNESS) )
print('对比度:', Video.get(cv2.CAP_PROP_CONTRAST) )
print('饱和度:', Video.get(cv2.CAP_PROP_SATURATION) )
print('色调:', Video.get(cv2.CAP_PROP_HUE) )
print('曝光度:', Video.get(cv2.CAP_PROP_EXPOSURE) )

宽: 640.0
高: 480.0
帧率: 30.0
亮度: 0.0
对比度: 15.0
饱和度: 36.0
色调: 0.0
曝光度: -5.0
